## Load Base Stable Diffusion Model

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash
import torch
import requests
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionInpaintPipeline

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
)
image = Image.open(
            BytesIO(
                requests.get(
                    "https://huggingface.co/lllyasviel/sd-controlnet-canny/resolve/main/images/bird_canny.png"
                ).content
            )
        )
prompt = "black dog"
pipe.to('cuda')

## Smash it!

In [ ]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 512
smasher_config['image_width'] = 512
smasher_config['version'] = '1.4'

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="img2img",
        api_key='your-api-key',
        model_config=None,
        smasher_config=AutoSmasherConfig.from_pretrained("CompVis/stable-diffusion-v1-4"),
        device='cuda',
    )

## Base Model Generation

In [ ]:
%%time
pipe(prompt=prompt, image=image).images[0].show()

## Smashed Model Generation

In [ ]:
smashed_model(prompt, init_image=image, num_inference_steps=50, image_height=768, image_width=512)[0]